In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
  
from statistics import mode

from scipy import stats

import numpy as np
import pandas as pd
import os
import glob
import math
import re
import enchant
import pickle
import time
import editdistance

/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
dict_label = {
    'Datetime':0, 
    'Sentence':1, 
    'Custom Object': 2, 
    'URL': 3, 
    'Numbers': 4, 
    'List': 5}

data = pd.read_csv('data/needs_extraction_data/labelled_added.csv')
data['y_act'] = [dict_label[i] for i in data['y_act']]
y = data.loc[:,['y_act']]
data.columns

Index(['Record_id', 'Attribute_name', 'y_pred', 'y_act', 'Reason',
       'total_vals', 'num_nans', '%_nans', 'mean_word_count',
       'std_dev_word_count', 'has_delimiters', 'sample_1', 'sample_2',
       'sample_3', 'sample_4', 'sample_5', 'mean_stopword_total',
       'mean_whitespace_count', 'mean_char_count', 'mean_delim_count',
       'stdev_stopword_total', 'stdev_whitespace_count', 'stdev_char_count',
       'stdev_delim_count', 'has_url', 'has_date'],
      dtype='object')

In [3]:
data1 = data[['%_nans', 'mean_word_count',
              'std_dev_word_count', 'has_delimiters', 'mean_stopword_total',
              'mean_whitespace_count', 'mean_char_count', 'mean_delim_count',
              'stdev_stopword_total', 'stdev_whitespace_count', 'stdev_char_count',
              'stdev_delim_count', 'has_url', 'has_date']]
# data1 = data1.fillna(0)

# data1 = data1.rename(columns={
#     'mean_word_count': 'scaled_mean_token_count',
#     'std_dev_word_count': 'scaled_std_dev_token_count',
#     '%_nans': 'scaled_perc_nans',
#     'mean_stopword_total': 'scaled_mean_stopword_total',
#     'mean_whitespace_count': 'scaled_mean_whitespace_count',
#     'mean_char_count': 'scaled_mean_char_count',
#     'mean_delim_count': 'scaled_mean_delim_count',
#     'stdev_stopword_total': 'scaled_stdev_stopword_total',
#     'stdev_whitespace_count': 'scaled_stdev_whitespace_count',
#     'stdev_char_count': 'scaled_stdev_char_count',
#     'stdev_delim_count': 'scaled_stdev_delim_count'
# })
# data1.loc[data1['scaled_mean_token_count'] >
#           10000, 'scaled_mean_token_count'] = 10000
# data1.loc[data1['scaled_mean_token_count'] < -
#           10000, 'scaled_mean_token_count'] = -10000

# data1.loc[data1['scaled_std_dev_token_count'] >
#           10000, 'scaled_std_dev_token_count'] = 10000
# data1.loc[data1['scaled_std_dev_token_count'] < -
#           10000, 'scaled_std_dev_token_count'] = -10000

# data1.loc[data1['scaled_perc_nans'] > 10000, 'scaled_perc_nans'] = 10000
# data1.loc[data1['scaled_perc_nans'] < -10000, 'scaled_perc_nans'] = -10000

# data1.loc[data1['scaled_mean_stopword_total'] >
#           10000, 'scaled_mean_stopword_total'] = 10000
# data1.loc[data1['scaled_mean_stopword_total'] < -
#           10000, 'scaled_mean_stopword_total'] = -10000

# data1.loc[data1['scaled_mean_whitespace_count'] >
#           10000, 'scaled_mean_whitespace_count'] = 10000
# data1.loc[data1['scaled_mean_whitespace_count'] < -
#           10000, 'scaled_mean_whitespace_count'] = -10000

# data1.loc[data1['scaled_mean_char_count'] >
#           10000, 'scaled_mean_char_count'] = 10000
# data1.loc[data1['scaled_mean_char_count'] < -
#           10000, 'scaled_mean_char_count'] = -10000

# data1.loc[data1['scaled_mean_delim_count'] >
#           10000, 'scaled_mean_delim_count'] = 10000
# data1.loc[data1['scaled_mean_delim_count'] < -
#           10000, 'scaled_mean_delim_count'] = -10000

# data1.loc[data1['scaled_stdev_stopword_total'] >
#           10000, 'scaled_stdev_stopword_total'] = 10000
# data1.loc[data1['scaled_stdev_stopword_total'] < -
#           10000, 'scaled_stdev_stopword_total'] = -10000

# data1.loc[data1['scaled_stdev_whitespace_count'] >
#           10000, 'scaled_stdev_whitespace_count'] = 10000
# data1.loc[data1['scaled_stdev_whitespace_count'] < -
#           10000, 'scaled_stdev_whitespace_count'] = -10000

# data1.loc[data1['scaled_stdev_char_count'] >
#           10000, 'scaled_stdev_char_count'] = 10000
# data1.loc[data1['scaled_stdev_char_count'] < -
#           10000, 'scaled_stdev_char_count'] = -10000

# data1.loc[data1['scaled_stdev_delim_count'] >
#           10000, 'scaled_stdev_delim_count'] = 10000
# data1.loc[data1['scaled_stdev_delim_count'] < -
#           10000, 'scaled_stdev_delim_count'] = -10000

# column_names_to_normalize = ['scaled_mean_token_count',
#                              'scaled_std_dev_token_count',
#                              'scaled_perc_nans',
#                              'scaled_mean_stopword_total',
#                              'scaled_mean_whitespace_count',
#                              'scaled_mean_char_count',
#                              'scaled_mean_delim_count',
#                              'scaled_stdev_stopword_total',
#                              'scaled_stdev_whitespace_count',
#                              'scaled_stdev_char_count',
#                              'scaled_stdev_delim_count']
columns_to_use = ['%_nans', 'mean_word_count',
              'std_dev_word_count', 'has_delimiters', 'mean_stopword_total',
              'mean_whitespace_count', 'mean_char_count', 'mean_delim_count',
              'stdev_stopword_total', 'stdev_whitespace_count', 'stdev_char_count',
              'stdev_delim_count', 'has_url', 'has_date']
x = data1[columns_to_use].values
x = np.nan_to_num(x)
x_scaled = StandardScaler().fit_transform(x)
df_temp = pd.DataFrame(
    x_scaled, columns=columns_to_use, index=data1.index)
data1[columns_to_use] = df_temp

y.y_act = y.y_act.astype(float)

print(f"> Data mean: {data1.mean()}\n")
print(f"> Data median: {data1.median()}\n")
print(f"> Data stdev: {data1.std()}")

print("===[VECTORIZATION]===")
arr = data['Attribute_name'].values
data = data.fillna(0)
arr1 = data['sample_1'].values
arr1 = [str(x) for x in arr1]
arr2 = data['sample_2'].values
arr2 = [str(x) for x in arr2]

vectorizer = CountVectorizer(ngram_range=(3, 3), analyzer='char')
X = vectorizer.fit_transform(arr)
X1 = vectorizer.fit_transform(arr1)
X2 = vectorizer.fit_transform(arr2)

print(f"> Length of vectorized feature_names: {len(vectorizer.get_feature_names())}")

# data1.to_csv('data/preprocessing/before.csv')
attr_df = pd.DataFrame(X.toarray())
sample1_df = pd.DataFrame(X1.toarray())
sample2_df = pd.DataFrame(X2.toarray())
# data2 = pd.concat([data1, attr_df, sample1_df, sample2_df], axis=1, sort=False)
data2 = pd.concat([data1, attr_df], axis=1, sort=False)

# data2.to_csv('data/preprocessing/after.csv')
data2.head()

X_train, X_test_held, y_train, y_test_held = train_test_split(
    data2, y, test_size=0.2, random_state=100)

X_train_new = X_train.reset_index(drop=True)
y_train_new = y_train.reset_index(drop=True)
print(f"X_train preview:\n{X_train.head()}")
print(f"y_train preview:\n{y_train.head()}")

X_train_new = X_train_new.values
y_train_new = y_train_new.values

/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/admin/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


> Data mean: %_nans                   -2.745801e-16
mean_word_count          -1.117919e-16
std_dev_word_count       -2.236863e-17
has_delimiters           -8.619107e-18
mean_stopword_total       8.619107e-18
mean_whitespace_count    -1.126127e-16
mean_char_count          -2.385646e-17
mean_delim_count         -5.915375e-17
stdev_stopword_total      6.413026e-17
stdev_whitespace_count   -2.236863e-17
stdev_char_count         -3.488686e-18
stdev_delim_count         9.516930e-17
has_url                  -1.327753e-16
has_date                  1.526813e-16
dtype: float64

> Data median: %_nans                   -0.653046
mean_word_count          -0.144106
std_dev_word_count       -0.171320
has_delimiters           -0.671120
mean_stopword_total      -0.178121
mean_whitespace_count    -0.144106
mean_char_count          -0.161318
mean_delim_count         -0.169718
stdev_stopword_total     -0.204091
stdev_whitespace_count   -0.171320
stdev_char_count         -0.188179
stdev_delim_count        

In [4]:
X_train_train, X_val, y_train_train, y_val = train_test_split(
    X_train_new, y_train_new, test_size=0.25, random_state=100)

In [14]:
def record_failed(model, x_test, y_test):
    data = pd.read_csv('data/needs_extraction_data/labelled_added.csv')
    dict_label = {
        0: 'Datetime',
        1: 'Sentence',
        2: 'Custom Object',
        3: 'URL',
        4: 'Numbers',
        5: 'List'
    }
    preds = model.predict(x_test)
    ret_df = pd.DataFrame(columns=['Index', 'Attribute_name', 'y_pred', 'y_act', 'sample_1', '%_nans', 'mean_word_count',
              'std_dev_word_count', 'has_delimiters', 'mean_stopword_total',
              'mean_whitespace_count', 'mean_char_count', 'mean_delim_count',
              'stdev_stopword_total', 'stdev_whitespace_count', 'stdev_char_count',
              'stdev_delim_count', 'has_url', 'has_date'])
    count = 0
    for pred in preds:
        y_true = int(y_test.values[count])
        if pred != y_true:
            index = x_test.index[count]
            row = data.loc[index]
            samples = [
                row.sample_1
            ]
            attr_name = row['Attribute_name']
            ret_df.loc[count] = [index, attr_name, pred, y_true] + samples + list(row[['%_nans', 'mean_word_count',
              'std_dev_word_count', 'has_delimiters', 'mean_stopword_total',
              'mean_whitespace_count', 'mean_char_count', 'mean_delim_count',
              'stdev_stopword_total', 'stdev_whitespace_count', 'stdev_char_count',
              'stdev_delim_count', 'has_url', 'has_date']])
        count += 1
    ret_df['y_act'] = ret_df['y_act'].astype(int)
    ret_df['y_pred'] = ret_df['y_pred'].astype(int)
    ret_df['y_act'] = [dict_label[i] for i in ret_df['y_act']]
    ret_df['y_pred'] = [dict_label[i] for i in ret_df['y_pred']]
    return ret_df


def get_false_nums(df):
    false_labels = df.y_pred.unique()
    true_labels = df.y_act.unique()
    length = len(df)
    for label in false_labels:
        total = len(df.loc[df.y_pred == label])
        print(f">>> Incorrectly predicted as {label}: {total}, {'{0:.3g}'.format((total/length)*100)}%")
        
        temp_df = df.loc[df.y_pred == label]
        temp_labels = temp_df.y_act.unique()
        t_len = len(temp_df)
        for t_label in temp_labels:
            t_total = len(temp_df[temp_df.y_act == t_label])
            print(f"\t>>> {t_label} predicted as {label}: {t_total}, {'{0:.5g}'.format((t_total/t_len)*100)}%")
    print("\n")

In [15]:
rf_clf = RandomForestClassifier(n_estimators=50, max_depth=50, random_state=100)
lr_clf = LogisticRegression(C=1, random_state=100)
svm_clf = svm.SVC(C=10, gamma=0.1, random_state=100)

rf_clf.fit(X_train_train, y_train_train)
lr_clf.fit(X_train_train, y_train_train)
svm_clf.fit(X_train_train, y_train_train)

print("="*20,"[Random Forest]","="*20)
trainsc = rf_clf.score(X_train, y_train)
valsc = rf_clf.score(X_val, y_val)
heldsc = rf_clf.score(X_test_held, y_test_held)
held_pr = metrics.precision_recall_fscore_support(y_test_held, rf_clf.predict(X_test_held))
print(f"> Train: {trainsc}")
print(f"> Validate: {valsc}")
print(f"> Test: {heldsc}")
print(f"> Precision Numbers: {held_pr[0][1]}")
print(f"> Precision Not Numbers: {held_pr[0][0]}")
print(f"> Recall Numbers: {held_pr[1][1]}")
print(f"> Recall Not Numbers: {held_pr[1][0]}")
print()
print("="*20,"[Logistic Regression]","="*20)
trainsc = lr_clf.score(X_train, y_train)
valsc = lr_clf.score(X_val, y_val)
heldsc = lr_clf.score(X_test_held, y_test_held)
held_pr = metrics.precision_recall_fscore_support(y_test_held, lr_clf.predict(X_test_held))
print(f"> Train: {trainsc}")
print(f"> Validate: {valsc}")
print(f"> Test: {heldsc}")
print(f"> Precision Numbers: {held_pr[0][1]}")
print(f"> Precision Not Numbers: {held_pr[0][0]}")
print(f"> Recall Numbers: {held_pr[1][1]}")
print(f"> Recall Not Numbers: {held_pr[1][0]}")
print()
print("="*20,"[SVM]","="*20)
trainsc = svm_clf.score(X_train, y_train)
valsc = svm_clf.score(X_val, y_val)
heldsc = svm_clf.score(X_test_held, y_test_held)
held_pr = metrics.precision_recall_fscore_support(y_test_held, svm_clf.predict(X_test_held))
print(f"> Train: {trainsc}")
print(f"> Validate: {valsc}")
print(f"> Test: {heldsc}")
print(f"> Precision Numbers: {held_pr[0][1]}")
print(f"> Precision Not Numbers: {held_pr[0][0]}")
print(f"> Recall Numbers: {held_pr[1][1]}")
print(f"> Recall Not Numbers: {held_pr[1][0]}")

/Users/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


==================== [Random Forest] ====================
> Train: 0.9537037037037037
> Validate: 0.8148148148148148
> Test: 0.8256880733944955
> Precision Numbers: 0.7692307692307693
> Precision Not Numbers: 0.8518518518518519
> Recall Numbers: 0.9090909090909091
> Recall Not Numbers: 0.8518518518518519

==================== [Logistic Regression] ====================
> Train: 0.9027777777777778
> Validate: 0.7777777777777778
> Test: 0.8348623853211009
> Precision Numbers: 0.8421052631578947
> Precision Not Numbers: 0.9230769230769231
> Recall Numbers: 0.7272727272727273
> Recall Not Numbers: 0.8888888888888888

==================== [SVM] ====================


/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


> Train: 0.9166666666666666
> Validate: 0.7314814814814815
> Test: 0.7981651376146789
> Precision Numbers: 1.0
> Precision Not Numbers: 0.92
> Recall Numbers: 0.6363636363636364
> Recall Not Numbers: 0.8518518518518519


In [16]:
rf_clf = RandomForestClassifier(n_estimators=50, max_depth=50, random_state=100)
lr_clf = LogisticRegression(C=1, random_state=100)
svm_clf = svm.SVC(C=10, gamma=0.1, random_state=100)

rf_clf.fit(X_train_train, y_train_train)
lr_clf.fit(X_train_train, y_train_train)
svm_clf.fit(X_train_train, y_train_train)

print("="*20,"[Random Forest]","="*20)
fails = record_failed(rf_clf, X_test_held, y_test_held)
print(f">>> Total incorrect predictions: {len(fails)}, {'{0:.4g}'.format(((len(fails)/len(y_test_held))*100))}%")
get_false_nums(fails)
print(fails.head(10))
fails.to_csv("rf_false.csv", index=False)
print("="*20,"[Logistic Regression]","="*20)
fails = record_failed(lr_clf, X_test_held, y_test_held)
print(f">>> Total incorrect predictions: {len(fails)}, {'{0:.4g}'.format(((len(fails)/len(y_test_held))*100))}%")
get_false_nums(fails)
print(fails.head(10))
fails.to_csv("lr_false.csv", index=False)
print("="*20,"[SVM]","="*20)
fails = record_failed(svm_clf, X_test_held, y_test_held)
print(f">>> Total incorrect predictions: {len(fails)}, {'{0:.4g}'.format(((len(fails)/len(y_test_held))*100))}%")
get_false_nums(fails)
print(fails.head(10))
fails.to_csv("svm_false.csv", index=False)

/Users/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


==================== [Random Forest] ====================
>>> Total incorrect predictions: 19, 17.43%
>>> Incorrectly predicted as Custom Object: 8, 42.1%
	>>> List predicted as Custom Object: 2, 25%
	>>> Datetime predicted as Custom Object: 4, 50%
	>>> Sentence predicted as Custom Object: 2, 25%
>>> Incorrectly predicted as Sentence: 6, 31.6%
	>>> List predicted as Sentence: 2, 33.333%
	>>> Custom Object predicted as Sentence: 4, 66.667%
>>> Incorrectly predicted as Datetime: 4, 21.1%
	>>> Custom Object predicted as Datetime: 3, 75%
	>>> Numbers predicted as Datetime: 1, 25%
>>> Incorrectly predicted as List: 1, 5.26%
	>>> Custom Object predicted as List: 1, 100%


   Index                              Attribute_name         y_pred  \
0    485                                   site_info  Custom Object   
1    188  customers_who_bought_this_item_also_bought       Sentence   
4    260                                    soloists       Sentence   
5    344                   ri_coll_teach_

>>> Total incorrect predictions: 22, 20.18%
>>> Incorrectly predicted as Custom Object: 17, 77.3%
	>>> List predicted as Custom Object: 5, 29.412%
	>>> Datetime predicted as Custom Object: 4, 23.529%
	>>> Sentence predicted as Custom Object: 8, 47.059%
>>> Incorrectly predicted as Datetime: 2, 9.09%
	>>> Numbers predicted as Datetime: 1, 50%
	>>> Custom Object predicted as Datetime: 1, 50%
>>> Incorrectly predicted as List: 2, 9.09%
	>>> Custom Object predicted as List: 2, 100%
>>> Incorrectly predicted as Numbers: 1, 4.55%
	>>> Custom Object predicted as Numbers: 1, 100%


   Index                              Attribute_name         y_pred  \
0    485                                   site_info  Custom Object   
1    188  customers_who_bought_this_item_also_bought  Custom Object   
8    477                               Period Ending  Custom Object   
9    175                             Survey Question  Custom Object   
15   360                                    duration       Datet